# 1. Import and Install Dependencies

In [1]:
%pip install tensorflow==2.10.0 opencv-python mediapipe scikit-learn matplotlib

ERROR: Could not find a version that satisfies the requirement tensorflow==2.10.0 (from versions: 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==2.10.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
import importlib # for reloading .py files
# WARNING: updates of config may require restart of kernel if reload is unsuccessful

import config
# reload config without restarting the kernel
importlib.reload(config)
from config import actions, no_sequences, sequence_length, DATA_PATH

import utils
# reload utils without restarting the kernel
importlib.reload(utils)
from utils import mediapipe_detection, extract_keypoints, draw_styled_landmarks, prob_viz


# 2. Keypoints using MP Holistic

In [4]:
from utils import mp_holistic, mp_drawing, set_camera_settings, default_fps

# 4. Setup Folders for Collection

In [6]:
empty_signs = ["background", "NoHands"]
actions = [
"ILoveYou",
"Yes",
"No",
"Hi",
"ThankYou",
"Me",
"You",
"It",
"Feel",
"Happy",
"Hungry",
"Eat",
"Bread",
"Chocolate",
"Like",
"Tired",
"Sleep",
"Evening",
"Bye",
"Bed",
]
actions = empty_signs + actions
actions

['background',
 'NoHands',
 'ILoveYou',
 'Yes',
 'No',
 'Hi',
 'ThankYou',
 'Me',
 'You',
 'It',
 'Feel',
 'Happy',
 'Hungry',
 'Eat',
 'Bread',
 'Chocolate',
 'Like',
 'Tired',
 'Sleep',
 'Evening',
 'Bye',
 'Bed']

In [7]:
# create folders

for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [8]:
import time

frame_count = 0
fps_list = []
start_time = time.time()


cap = cv2.VideoCapture(1)
actual_res, actual_fps = set_camera_settings(cap) # setting camera resolution and fps + printing the actual values

# Create a named window and resize it

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # Flag variable to signal breaking out of all loops
    break_all_loops = False

    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Check if flag variable is set to break all loops
                if break_all_loops:
                    break

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                # print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # NEW Apply wait logic
                # Display messages and countdown
                # Apply wait logic with countdown
                if frame_num == 0:
                    for countdown in range(2, 0, -1):  # 3 seconds countdown
                        cv2.putText(image, f'GET READY FOR "{action}" in {countdown}', (30,100),
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0, 0), 4, cv2.LINE_AA)
                        # cv2.putText(image, f'Collecting frames for {action} Video Number {sequence+1}', (15,150),
                        #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                        cv2.imshow('OpenCV Feed', image)
                        cv2.waitKey(1000)  # wait 1000 ms between each number
                        image = frame.copy()  # clear the previous text
                else:
                    cv2.putText(image, f'RECORDING [{sequence + 1}/{no_sequences}] "{action}" - Frame {frame_num + 1}/{sequence_length}', (30,100),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0, 255), 4, cv2.LINE_AA)
                    remaining_frames = sequence_length - frame_num - 1
                    cv2.imshow('OpenCV Feed', image)

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                print(npy_path)
                np.save(npy_path, keypoints)

                frame_count += 1

                if frame_count % 100 == 0:
                    end_time = time.time()
                    elapsed = end_time - start_time
                    actual_fps = frame_count / elapsed
                    fps_list.append(actual_fps)
                    print("Actual FPS:", actual_fps)
                    # Reset timer and counter
                    start_time = time.time()
                    frame_count = 0

                # Break gracefully - go to next frame
                if cv2.waitKey(10) & 0xFF == ord('t'):
                    # Set flag variable to break out of all loops
                    break

                # Break gracefully - kill all
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    # Set flag variable to break out of all loops
                    break_all_loops = True
                    break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

Requested Resolution: 640x480, Actual Resolution: 640x480
Requested FPS: 30, Actual FPS: 30.0


I0000 00:00:1715687038.629347       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


MP_Data/background/0/0
MP_Data/background/0/1
MP_Data/background/0/2
MP_Data/background/0/3
MP_Data/background/0/4
MP_Data/background/0/5
MP_Data/background/0/6
MP_Data/background/0/7
MP_Data/background/0/8
MP_Data/background/0/9
MP_Data/background/0/10
MP_Data/background/0/11
MP_Data/background/0/12
MP_Data/background/0/13
MP_Data/background/0/14
MP_Data/background/0/15
MP_Data/background/0/16
MP_Data/background/0/17
MP_Data/background/0/18
MP_Data/background/0/19
MP_Data/background/0/20
MP_Data/background/0/21
MP_Data/background/0/22
MP_Data/background/0/23
MP_Data/background/0/24
MP_Data/background/0/25
MP_Data/background/0/26
MP_Data/background/0/27
MP_Data/background/0/28
MP_Data/background/0/29
MP_Data/background/1/0
MP_Data/background/1/1
MP_Data/background/1/2
MP_Data/background/1/3
MP_Data/background/1/4
MP_Data/background/1/5
MP_Data/background/1/6
MP_Data/background/1/7
MP_Data/background/1/8
MP_Data/background/1/9
MP_Data/background/1/10
MP_Data/background/1/11
MP_Data/back

In [11]:
fps_list

[6.0823465378505315, 7.501147401009512, 7.178792146737843]

In [2]:
fps_check = [5.445442143436572,
 7.249745231581113,
 6.290899129609047,
 5.955219709672702,
 6.8669062454946355,
 8.658761533004986,
 8.171947604739815]

In [3]:
sum(fps_check) / len(fps_check)

6.9484173710769825

In [ ]:
cap.release()
cv2.destroyAllWindows()